# The ``attrs`` library

## ``attrs`` by example

https://www.attrs.org/en/stable/examples.html

### Basic usage

#### Without attributes

In [52]:
import attr

In [53]:
@attr.s
class Empty(object):
    pass

In [54]:
Empty()

Empty()

In [55]:
Empty() == Empty()

True

In [56]:
Empty() is Empty()

False

#### With simple attributes

In [57]:
@attr.s
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()

In [58]:
c1 = Coordinates(1, 2)
c1

Coordinates(x=1, y=2)

In [59]:
c2 = Coordinates(x=2, y=1)
c2

Coordinates(x=2, y=1)

In [60]:
c1 == c2

False

In [61]:
Coordinates.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Coordinates' objects>,
              '__weakref__': <attribute '__weakref__' of 'Coordinates' objects>,
              '__doc__': None,
              '__attrs_attrs__': (Attribute(name='x', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False, inherited=False, on_setattr=None),
               Attribute(name='y', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False, inherited=False, on_setattr=None)),
              '__repr__': <function __main__.Coordinates.__repr__(self)>,
              '__eq__': <function __main__.Coordinates.__eq__(self, other)>,
              '__ne__': <function __main__.Coordinates.__ne__(self, other)>,
              '

In [62]:
from attr import attrs, attrib
# Also serious naming
@attrs
class SeriousCoordinates:
    x = attrib()
    y = attrib()

In [63]:
SeriousCoordinates(1, 2)

SeriousCoordinates(x=1, y=2)

In [64]:
attr.fields(Coordinates) == attr.fields(Coordinates)

True

#### Private attributes

In [65]:
@attr.s
class C:
    _x = attr.ib()

In [66]:
C(x=1)

C(_x=1)

In [67]:
# Default argument
@attr.s
class C:
    _x = attr.ib(init=False, default=42)

In [68]:
# Throws error
# C(23) 

C()

C(_x=42)

In [69]:
class SomethingFromSomeoneElse:
    def __init__(self, x):
        self.x = x
        
SomethingFromSomeoneElse = attr.s(
    these = {
        "x": attr.ib()
    }, init=False)(SomethingFromSomeoneElse)

In [70]:
SomethingFromSomeoneElse(10)

SomethingFromSomeoneElse(x=10)

#### Nested classes

In [71]:
@attr.s
class C(object):
    @attr.s(repr_ns="C")
    class D(object):
        pass

In [72]:
C.D()

C.D()

#### Keyword only attributes

In [73]:
@attr.s
class A:
    a = attr.ib(kw_only=True)

In [74]:
# VVV Throws error
# A()

A(a = 1)

A(a=1)

In [75]:
@attr.s(kw_only=True)
class A:
    a = attr.ib()
    b = attr.ib()

In [76]:
# VVV Throws error
# A(1, 2)

A(a=1, b=2)

A(a=1, b=2)

In [77]:
@attr.s(kw_only=True)
class B:
    a = attr.ib(init=False)
    b = attr.ib()

# VVV Throws error
# A(a=1, b=2)

B(b=2)

B(a=NOTHING, b=2)

### Converting to collections types

#### Converting to dict

In [78]:
attr.asdict(Coordinates(x=1, y=2))

{'x': 1, 'y': 2}

In [79]:
# Excluding fields

@attr.s
class UserList:
    users = attr.ib()

@attr.s
class User:
    email = attr.ib()
    password = attr.ib()

attr.asdict(UserList([User("jane@doe.invalid", "s33kred"),
                      User("joe@doe.invalid", "p4ssw0rd")]),
            filter=lambda attr, value: attr.name != "password")

{'users': [{'email': 'jane@doe.invalid'}, {'email': 'joe@doe.invalid'}]}

In [80]:
@attr.s
class User:
    login = attr.ib()
    password = attr.ib()
    id = attr.ib()

attr.asdict(User("jane", "s33kred", 42),
            filter=attr.filters.exclude(attr.fields(User).password, int)
           )

{'login': 'jane'}

In [81]:
@attr.s
class C:
    x = attr.ib()
    y = attr.ib()
    z = attr.ib()

attr.asdict(C("foo", "2", 3),
           filter=attr.filters.include(int, attr.fields(C).x))

{'x': 'foo', 'z': 3}

#### Converting to tuple

In [82]:
import sqlite3
import attr

@attr.s
class Foo:
    a = attr.ib()
    b = attr.ib()
    
foo = Foo(2, 3)

with sqlite3.connect(":memory:") as conn:
    c = conn.cursor()
    c.execute("CREATE TABLE foo (x INTEGER PRIMARY KEY ASC, y)") 
    c.execute("INSERT INTO foo VALUES (?, ?)", attr.astuple(foo)) 
    foo2 = Foo(*c.execute("SELECT x, y FROM foo").fetchone())
    
print(foo)
print(foo2)

OperationalError: table foo already exists

In [ ]:
foo == foo2

### Defaults

In [83]:
import collections

@attr.s
class Connection:
    socket = attr.ib()
    @classmethod
    def connect(cls, db_string):
        # ... get connection
        return cls(socket=42)

@attr.s
class ConnectionPool:
    db_string = attr.ib()
    pool = attr.ib(default=attr.Factory(collections.deque))
    debug = attr.ib(default=False)
    
    def get_connection(self):
        try:
            return self.pool.pop()
        except IndexError:
            if self.debug:
                print("New connection!")
            
            return Connection.connect(self.db_string)
    
    def free_connection(self, conn):
        if self.debug:
            print("Connection returned")
            
        self.pool.appendleft(conn)

In [84]:
cp = ConnectionPool("postgres://localhost")
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([]), debug=False)

In [85]:
conn = cp.get_connection()
conn

Connection(socket=42)

In [86]:
cp.free_connection(conn)
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([Connection(socket=42)]), debug=False)

#### Default decorator

In [87]:
@attr.s
class C:
    x = attr.ib(default=1)
    y = attr.ib()
    
    @y.default
    def  _any_name_exept_a_name_of_an_attribute(self):
        return self.x + 1
    
C()

C(x=1, y=2)

In [88]:
@attr.s
class C:
    # Sugar for attr.ib(default=attr.Factory(f))
    x = attr.ib(factory=list)

C()

C(x=[])

### Validators

> [...] the validators runs *after* the instance is initialized, you can refer to other attributes while validating:

In [90]:
# Using a decorator

@attr.s
class C:
    x = attr.ib()
    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("x must be smaller or equal to 42")
            
C(42)
# C(43) <-- raises an error

C(x=42)

In [99]:
# Callable

def x_smaller_than_y(instance, attribute, value):
    if value >= instance.y:
        raise ValueError("'x' has to be smaller than 'y'")
        
        
@attr.s
class C:
    x = attr.ib(validator=[attr.validators.instance_of(int), x_smaller_than_y])
    y = attr.ib()
  
# C(x=4, y=3) <-- raises an error

C(x=3, y=4)

C(x=3, y=4)

### Converters

In [103]:
@attr.s
class C:
    x = attr.ib(converter=int)
    
o = C("1")
o.x

1

https://www.attrs.org/en/stable/init.html#converters

### Types

In [113]:
@attr.s
class C:
    x = attr.ib(type=int)
    y : int = attr.ib()

In [110]:
attr.fields(C).x.type

int

In [112]:
attr.fields(C).y.type

None


In [118]:
import typing

@attr.s(auto_attribs=True)
class AutoC:
    cls_var: typing.ClassVar[int] = 5
    l: typing.List[int] = attr.Factory(list)
    x: int = 1
    foo : str = attr.ib(
        default="every attrib needs a type if auto_attrib=True"
    )
    bar: typing.Any = None

In [119]:
attr.fields(AutoC).l.type

typing.List[int]

In [120]:
attr.fields(AutoC).x.type

int

In [121]:
attr.fields(AutoC).foo.type

str

In [122]:
attr.fields(AutoC).bar.type

typing.Any

In [116]:
AutoC()

AutoC(l=[], x=1, foo='every attrib needs a type if auto_attrib=True', bar=None)

In [123]:
AutoC.cls_var

5

### Typing

In [126]:
import attr
import typing

@attr.s(auto_attribs=True)
class SomeClass:
    a_number : int = 42
    list_of_numbers: typing.List[int] = attr.Factory(list)

In [128]:
sc = SomeClass(1, [1, 2, 3])
sc

SomeClass(a_number=1, list_of_numbers=[1, 2, 3])

In [130]:
attr.fields(SomeClass).a_number.type

int

In [144]:
@attr.s
class Point:
    x : float = attr.ib(
        converter=float
    )
    
Point("33")

Point(x=33.0)